In [ ]:
import mrcnn
from mrcnn.config import Config
import mrcnn.model
import mrcnn.visualize
import cv2
import random
import re
import pandas as pd
import numpy as np
import math
import bisect
import time
import statistics
import matplotlib
import matplotlib.pyplot as plt
import pylab

In [12]:

df=pd.read_csv(r"C:\Users\logan\Desktop\MEng\Mask_RCNN\.rawForcedata\12p5m_9ths_0p5kts_0p6m_0deg_001_Processed_MS_50Hz.csv")
temp=df.values.tolist()[0]

print(temp)

[0, 0.0004, -9.817671428999999, -0.727092857, 9.844985714285713, 1.6574785714285711, '12p5', '9ths', '0p5kts', '0p6m', 0, 1]
